In [146]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pylab
import matplotlib.pyplot as plt
import statsmodels.tsa.seasonal as seasonal
import statsmodels as sm

In [147]:
df = pd.read_excel("case_study_ML.xlsx")

In [148]:
df.ISO_Week = pd.to_datetime(df.ISO_Week.add('-0'), format='%Y-%W-%w')
df.set_index('ISO_Week',inplace = True)
df.head()

,SKU,Sales,Season
ISO_Week,,,
2018-01-07,colgate,0.0,WINTER
2018-01-14,colgate,0.0,WINTER
2018-01-21,colgate,0.0,WINTER
2018-01-28,colgate,6988.0,WINTER
2018-01-28,colgate,6988.0,WINTER


In [149]:
exog_var = pd.read_csv("exog_var.csv")
exog_var.Weeks = pd.to_datetime(exog_var.Weeks.add('-0'), format='%Y-%W-%w')
exog_var.rename(columns = {'Weeks':'ISO_Week'},inplace=True)
exog_var.set_index('ISO_Week',inplace = True)
exog_var.head()

,SKU
ISO_Week,
2018-03-18,colgate
2018-03-25,colgate
2018-04-01,colgate
2018-04-08,colgate
2018-04-15,colgate


In [150]:
exog_var['offer'] = 1
exog_var.head()

,SKU,offer
ISO_Week,,
2018-03-18,colgate,1
2018-03-25,colgate,1
2018-04-01,colgate,1
2018-04-08,colgate,1
2018-04-15,colgate,1


In [151]:
x=pd.to_datetime('2018-42-0',format='%Y-%W-%w')
x

Timestamp('2018-10-21 00:00:00')

## Colgate

In [152]:
df_colgate = df[df.SKU == 'colgate']
exog_var_colgate = exog_var[exog_var.SKU=='colgate']
exog_var_colgate.drop('SKU',axis =1,inplace=True)
data_colgate = pd.merge(df_colgate,exog_var_colgate,on='ISO_Week',how = 'left')
data_colgate['offer'].fillna(0,inplace = True)
data_colgate.head()

,SKU,Sales,Season,offer
ISO_Week,,,,
2018-01-07,colgate,0.0,WINTER,0.0
2018-01-14,colgate,0.0,WINTER,0.0
2018-01-21,colgate,0.0,WINTER,0.0
2018-01-28,colgate,6988.0,WINTER,0.0
2018-01-28,colgate,6988.0,WINTER,0.0


#### Creating Train Set

In [153]:
data_colgate_train = data_colgate.loc[:'2018-10-20',:]

#### Calculate Mean

In [154]:
colgate_sales_mean = data_colgate_train[data_colgate_train.Sales>0].Sales.mean(skipna=True)

#### Outlier Treatment

In [155]:
data_colgate_train = data_colgate_train[np.abs(data_colgate_train.Sales-data_colgate_train.Sales.mean())<=(3*data_colgate_train.Sales.std())]

#### NA and less than 0 Value cleaning

In [156]:
data_colgate_train.fillna(colgate_sales_mean,inplace = True)

In [157]:
data_colgate_train = data_colgate_train[data_colgate_train.Sales>0]

In [158]:
data_colgate_train.head()

,SKU,Sales,Season,offer
ISO_Week,,,,
2018-01-28,colgate,6988.0,WINTER,0.0
2018-01-28,colgate,6988.0,WINTER,0.0
2018-02-04,colgate,6743.0,WINTER,0.0
2018-02-11,colgate,4112.0,WINTER,0.0
2018-02-18,colgate,5732.0,WINTER,0.0


In [159]:
exog_var_forecast = data_colgate.loc['2018-10-20':,:]
exog_var_forecast = exog_var_forecast[['offer']]
exog_var_forecast.head()

,offer
ISO_Week,
2018-10-21,0.0
2018-10-28,0.0
2018-11-04,0.0
2018-11-11,1.0
2018-11-18,0.0


In [160]:
exog_var_train = data_colgate_train.loc[:'2018-10-20',:]
exog_var_train = exog_var_train[['offer']]
exog_var_train.head()

,offer
ISO_Week,
2018-01-28,0.0
2018-01-28,0.0
2018-02-04,0.0
2018-02-11,0.0
2018-02-18,0.0


#### Fitting Sarimax

In [161]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(data_colgate_train[['Sales']],exog=exog_var_train[['offer']],
    order=(0, 1, 0),
    seasonal_order=(0, 1, 0, 12),
    enforce_stationarity=True,
    enforce_invertibility=False)

/home/jupyterlab/conda/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [162]:
SARIMAX_results = model.fit()

#### Forecasting

In [163]:
SARIMAX_forecast=round(SARIMAX_results.forecast(steps = 11,exog = exog_var_forecast[['offer']]),2)

/home/jupyterlab/conda/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:531: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


In [164]:
colgate_forecast = pd.DataFrame(SARIMAX_forecast)
colgate_forecast.columns=['Sales']

In [165]:
data_colgate_forecast = data_colgate.loc['2018-10-20':,:]
data_colgate_forecast['forecast']=colgate_forecast['Sales'].values
data_colgate_forecast.index= data_colgate_forecast.index.strftime('%Y-%U')
data_colgate_forecast.to_csv('colgate_forecast.csv')

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Pepsodent

In [166]:
df_pepsodent = df[df.SKU == 'pepsodent']
exog_var_pepsodent = exog_var[exog_var.SKU=='pepsodent']
exog_var_pepsodent.drop('SKU',axis =1,inplace=True)
data_pepsodent = pd.merge(df_pepsodent,exog_var_pepsodent,on='ISO_Week',how = 'left')
data_pepsodent['offer'].fillna(0,inplace = True)
data_pepsodent.head()

,SKU,Sales,Season,offer
ISO_Week,,,,
2018-01-07,pepsodent,NaN,WINTER,0.0
2018-01-14,pepsodent,NaN,WINTER,0.0
2018-01-21,pepsodent,NaN,WINTER,0.0
2018-01-28,pepsodent,-23.0,WINTER,0.0
2018-02-04,pepsodent,446.0,WINTER,0.0


#### Creating Train Set

In [167]:
data_pepsodent_train = data_pepsodent.loc[:'2018-10-20',:]

#### Calculating Mean

In [168]:
pepsodent_sales_mean = data_pepsodent_train[data_pepsodent_train.Sales>0].Sales.mean(skipna=True)

#### Outlier Treatment

In [169]:
data_pepsodent_train = data_pepsodent_train[np.abs(data_pepsodent_train.Sales-data_pepsodent_train.Sales.mean())<=(3*data_pepsodent_train.Sales.std())]

#### NA and less than 0 Value Cleaning

In [170]:
data_pepsodent_train.fillna(pepsodent_sales_mean,inplace = True)

In [171]:
data_pepsodent_train = data_pepsodent_train[data_pepsodent_train.Sales>0]

In [172]:
exog_var_forecast = data_pepsodent.loc['2018-10-20':,:]
exog_var_forecast = exog_var_forecast[['offer']]
exog_var_forecast.head()

,offer
ISO_Week,
2018-10-21,0.0
2018-10-28,0.0
2018-11-04,0.0
2018-11-11,0.0
2018-11-18,0.0


In [173]:
exog_var_train = data_pepsodent_train.loc[:'2018-10-20',:]
exog_var_train = exog_var_train[['offer']]
exog_var_train.head()

,offer
ISO_Week,
2018-02-04,0.0
2018-02-11,0.0
2018-02-18,0.0
2018-02-25,0.0
2018-02-25,0.0


#### Fitting SARIMAX

In [174]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(data_pepsodent_train[['Sales']],exog=exog_var_train[['offer']],
    order=(0, 1, 0),
    seasonal_order=(0, 1, 0, 12),
    enforce_stationarity=True,
    enforce_invertibility=False)

/home/jupyterlab/conda/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [175]:
SARIMAX_results = model.fit()

#### Forecasting

In [176]:
SARIMAX_forecast=round(SARIMAX_results.forecast(steps = 11,exog = exog_var_forecast[['offer']]),2)

/home/jupyterlab/conda/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:531: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


In [177]:
pepsodent_forecast = pd.DataFrame(SARIMAX_forecast)
pepsodent_forecast.columns=['Sales']

In [178]:
data_pepsodent_forecast = data_pepsodent.loc['2018-10-20':,:]
data_pepsodent_forecast['forecast']=pepsodent_forecast['Sales'].values
data_pepsodent_forecast.index= data_pepsodent_forecast.index.strftime('%Y-%U')
data_pepsodent_forecast.to_csv('pepsodent_forecast.csv')

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## CloseUp

In [179]:
df_closeup = df[df.SKU == 'closeup']
exog_var_closeup = exog_var[exog_var.SKU=='closeup']
exog_var_closeup.drop('SKU',axis =1,inplace=True)
data_closeup = pd.merge(df_closeup,exog_var_closeup,on='ISO_Week',how = 'left')
data_closeup.drop_duplicates(inplace=True)
data_closeup['offer'].fillna(0,inplace = True)
data_closeup.head()

,SKU,Sales,Season,offer
ISO_Week,,,,
2018-03-18,closeup,5495.0,SPRING,0.0
2018-03-25,closeup,6330.0,SPRING,0.0
2018-04-01,closeup,6144.0,SPRING,0.0
2018-04-08,closeup,6383.0,SPRING,0.0
2018-04-15,closeup,5533.0,SPRING,0.0


#### Creating Train Set

In [180]:
data_closeup_train = data_closeup.loc[:'2018-10-20',:]

#### Calculating Mean

In [181]:
closeup_sales_mean = data_closeup_train[data_closeup_train.Sales>0].Sales.mean(skipna=True)

#### Outlier Treatment

In [182]:
data_closeup_train = data_closeup_train[np.abs(data_closeup_train.Sales-data_closeup_train.Sales.mean())<=(3*data_closeup_train.Sales.std())]



#### NA and less than 0 Value Cleaning

In [183]:
data_closeup_train.fillna(closeup_sales_mean,inplace = True)

In [184]:
data_closeup_train = data_closeup_train[data_closeup_train.Sales>0]

In [185]:
exog_var_forecast = data_closeup.loc['2018-10-20':,:]
exog_var_forecast = exog_var_forecast[['offer']]
exog_var_forecast.head()

,offer
ISO_Week,
2018-10-21,0.0
2018-10-28,0.0
2018-11-04,0.0
2018-11-11,0.0
2018-11-18,1.0


In [186]:
exog_var_train = data_closeup_train.loc[:'2018-10-20',:]
exog_var_train = exog_var_train[['offer']]
exog_var_train.head()

,offer
ISO_Week,
2018-03-18,0.0
2018-03-25,0.0
2018-04-01,0.0
2018-04-08,0.0
2018-04-15,0.0


#### Fitting SARIMAX

In [194]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
model = SARIMAX(data_closeup_train[['Sales']],exog=exog_var_train[['offer']],
    order=(0, 1, 0),
    seasonal_order=(0, 1, 0, 12),
    enforce_stationarity=True,
    enforce_invertibility=False)

/home/jupyterlab/conda/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:225: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)


In [195]:
SARIMAX_results = model.fit()

#### Forecasting

In [196]:
closeup_forecast = pd.DataFrame(SARIMAX_forecast)
closeup_forecast.columns=['Sales']

In [197]:
SARIMAX_forecast=round(SARIMAX_results.forecast(steps = 11,exog = exog_var_forecast[['offer']]),2)

/home/jupyterlab/conda/lib/python3.6/site-packages/statsmodels/tsa/base/tsa_model.py:531: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


In [198]:
data_closeup_forecast = data_closeup.loc['2018-10-20':,:]
data_closeup_forecast['forecast']=closeup_forecast['Sales'].values
data_closeup_forecast.index= data_closeup_forecast.index.strftime('%Y-%U')
data_closeup_forecast.to_csv('closeup_forecast.csv')

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Consolidation

In [199]:
forecast = pd.concat([data_colgate_forecast,data_pepsodent_forecast,data_closeup_forecast],axis = 0)

In [200]:
forecast

,SKU,Sales,Season,offer,forecast
2018-42,colgate,4740.0,AUTUMN,0.0,4759.00
2018-43,colgate,4341.0,AUTUMN,0.0,4423.00
2018-44,colgate,5363.0,AUTUMN,0.0,6624.00
2018-45,colgate,12120.0,AUTUMN,1.0,9238.19
2018-46,colgate,3314.0,AUTUMN,0.0,2880.00
2018-47,colgate,4673.0,AUTUMN,0.0,3269.00
2018-48,colgate,12376.0,AUTUMN,1.0,10569.19
2018-49,colgate,14853.0,WINTER,1.0,8705.19
2018-50,colgate,12447.0,WINTER,1.0,9312.19
2018-51,colgate,12247.0,WINTER,1.0,12135.19


In [201]:
forecast.to_csv('forecast.csv')